## Summary

The purpose of this notebook is to give you a better understanding of how the infrastructure deployed by CDK transcribes a file and summarizes it, using S3, Transcribe, Lambda, and Bedrock. At the end of this notebook you will get back a transcription (as `json` and `txt`) and summarization (using Bedrock) of your audio file. 

## Prerequisites

This notebook assumes that you have an AWS account, and sufficient IAM credentials to access Amazon S3, AWS Lambda, Amazon Transcribe, and Amazon Bedrock. It also assumes that you've already used the AWS CDK to deploy your project infrastructure. If you haven't done this yet, follow the instructions provided in [`README.md`](https://github.com/aws-samples/amazon-bedrock-audio-summarizer/blob/main/README.md).

Note: The [summarizer Lambda function](/lambda/eventbridge-bedrock-inference/lambda_function.py) deployed by the CDK is hardcoded to use Anthropic's Claude 3 Sonnet LLM. You can [enable access to Claude 3](https://console.aws.amazon.com/bedrock/home?#/models) via the AWS Bedrock Console, or replace the model ID and invocation parameters inside the Lambda function. 

## List S3 Buckets
Start by installing `boto3`:

In [ ]:
!pip install boto3

Then retrieve all of the available S3 buckets in your account. You should have a bucket that looks something like this: `summarizerstack-summarizer...`

In [ ]:
import boto3

bucket_name = ''
s3 = boto3.client('s3')
response = s3.list_buckets()
buckets = [bucket['Name'] for bucket in response['Buckets']]
for bucket in buckets:
    if bucket.startswith('summarizerstack-'):
        bucket_name = bucket
        print(f'Found bucket {bucket_name}')

if bucket_name == '':
    print('Summarizer bucket not found. Did you deploy the infrastructure with `cdk-deploy`?')

## Get folders

Update the code block below with your S3 bucket. After running this block, you should see three folders: `processed`, `source`, and `transcription`. This ensures that the app was deployed correctly. 

In [ ]:
bucket_folders = s3.list_objects_v2(Bucket=bucket_name, Delimiter='/')
print(f'Folders in {bucket_name}:')
for prefix in bucket_folders.get('CommonPrefixes', list()):
    print('\t - ' + prefix.get('Prefix', ''))

## Upload audio to S3

Next, upload an audio file to your S3 bucket in the `source` folder. When complete, this will trigger a series of Lambdas to transcribe and summarize the audio. In the code blow below, add your audio file name. **Note**: This example assumes that audio is in the current working directory. 

Supported [media formats](https://docs.aws.amazon.com/transcribe/latest/dg/how-input.html#how-input-audio): AMR, FLAC, M4A, MP3, MP4, Ogg, WebM, WAV. 


In [ ]:
import os

# Replace with your audio file. This assumes the file is in the current working directory.
audio_file_name = '<AUDIO_FILE>'   
file_path = os.path.join(os.getcwd(), audio_file_name)

# Upload the file to the S3 bucket
object_name = 'source/' + audio_file_name
with open(file_path, 'rb') as file:
    s3.upload_fileobj(file, bucket_name, object_name)
    print(f"File '{file_path}' uploaded to '{bucket_name}/{object_name}'")

## Monitor transcription status

When an audio file is uploaded to the `source` folder, an S3 trigger invoked a [Lambda function](/lambda/s3-trigger-transcribe/lambda_function.py). That Lambda function created an Amazon Transcribe job

The code block below is doing a few things: 

1. Checking for active transcriptions
2. Assigning the latest transcription to `job_name`
3. Monitoring the status of the transcription job

Depending on the size of the audio file, jobs can take a few minutes.   

In [ ]:
import time

# Create a Transcribe client
transcribe = boto3.client('transcribe')

# List active transcription jobs
try:
    response = transcribe.list_transcription_jobs(
        Status='IN_PROGRESS'
    )
    active_jobs = response['TranscriptionJobSummaries']

    # Sort active jobs by creation time
    active_jobs.sort(key=lambda job: job['CreationTime'], reverse=True)

    # Print the list of active jobs
    if active_jobs:
        print("Active transcription jobs:")
        for job in active_jobs:
            print(f"- {job['TranscriptionJobName']} ({job['TranscriptionJobStatus']})")
        
        # Assign the latest job name to job_name
        job_name = active_jobs[0]['TranscriptionJobName']
        print(f"\nThe latest transcription job is: {job_name}\n")
    else:
        print("No active transcription jobs found.")
        
except transcribe.exceptions.BadRequestException as e:
    print(f"Error: {e}")
except transcribe.exceptions.InternalFailureException as e:
    print(f"Error: {e}")
except transcribe.exceptions.LimitExceededException as e:
    print(f"Error: {e}")

max_retries = 60  # Maximum number of retries
retry_delay = 15  # Delay between retries (in seconds)

# Monitor/poll for transcription status
retries = 0
while retries < max_retries:
    try:
        response = transcribe.get_transcription_job(TranscriptionJobName=job_name)
        job_status = response['TranscriptionJob']['TranscriptionJobStatus']
        print(f"Job status: {job_status}")
        
        if job_status == 'COMPLETED':
            transcription_file_uri = response['TranscriptionJob']['Transcript']['TranscriptFileUri']
            print(f"Transcription file: {transcription_file_uri}")
            break
        elif job_status == 'FAILED':
            failure_reason = response['TranscriptionJob']['FailureReason']
            print(f"Job failed: {failure_reason}")
            break
        else:
            print(f"Job is still in progress. Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
            retries += 1

    except transcribe.exceptions.BadRequestException as e:
        print(f"Error: {e}")
    except transcribe.exceptions.InternalFailureException as e:
        print(f"Error: {e}")
    except transcribe.exceptions.LimitExceededException as e:
        print(f"Error: {e}")

## Retrieve the summary

Last but not least, let's get the transcription summary. Amazon EventBridge was watching for any job named `summarizer-` to reach a `COMPLETED` state. When it found one, it kicked off a [Lambda function](/lambda/eventbridge-bedrock-inference/lambda_function.py) to format the transcription and create an inference request to Amazon Bedrock.

The code block below is checking to see if there is a summary that matches the transcription job. If there is a match, the summary is printed below.

**Note:** It may take a few seconds for the summarization job to appear. 

In [ ]:
prefix = 'processed/'

# Call the list_objects_v2 method with the Prefix parameter
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

summary_found = False
while not summary_found:
    for obj in response.get('Contents', []):
        if job_name in obj['Key']:
            summary_found = True
            try:
                # Get the object from S3
                obj = s3.get_object(Bucket=bucket_name, Key=f'{prefix}{job_name}.txt')
            
                # Read the contents of the file
                summary = obj['Body'].read().decode('utf-8')
            
                # Print the summary
                print(summary)
            
            except Exception as e:
                print(f"An error occurred: {e}")
    if summary_found:
        break  # Exit the outer loop if the summary is found

## Clean up

The last step is to clean up your project using the CDK CLI. Follow the instructions in our [`README.md`](https://github.com/aws-samples/amazon-bedrock-audio-summarizer/blob/main/README.md).